# SHFQA

Just like the driver for the HDAWG in the previous example, we now use the `tk.SHFQA` instrument driver.

In [1]:
import zhinst.toolkit as tk

shfqa = tk.SHFQA("shfqa", "dev12036", interface="1gbe", host="localhost")
shfqa.setup()           # set up data server connection
shfqa.connect_device()  # connect device to data server

Successfully connected to data server at localhost:8004 api version: 6
Successfully connected to device DEV12036 on interface 1GBE


## Channel parameters of the SHFQA

SHFQA has 2 or 4 channels, each of them can serve for a single readout line. Each QAchannel module can readout up to 8 or 16 quits.

In [2]:
shfqa.qachannels[0]

In [3]:
shfqa.qachannels[0].input_range(0)
shfqa.qachannels[0].output_range(-15)
shfqa.qachannels[0].center_freq(5e9)
shfqa.qachannels[0].input('on')
shfqa.qachannels[0].output('on')

## Spectroscopy mode of the SHFQA
SHFQA has two application modes, **Spectroscopy** and **Readout**. Spectroscopy mode is generally used for resonator spectroscopy experiments, and Readout mode is used for qubit readout experiments with fixed readout frequencies. 

In Spectroscopy mode, offset frequency sweep is done by the **sweeper** module. Each frequency sweep is triggered by an internal or external trigger. External trigger is recommended. With **shfqa.set_trigger_loopback()** a marker output is connected to a trigger input internally without any physical connections. 

In [4]:
shfqa.qachannels[0].sweeper

In [5]:
shfqa.qachannels[0].mode('spectroscopy')
sweeper0=shfqa.qachannels[0].sweeper

shfqa.set_trigger_loopback()
sweeper0.trigger_source("channel0_trigger_input0")
sweeper0.trigger_level(0)
sweeper0.trigger_imp50(1)

sweeper0.start_frequency(-200e-6)
sweeper0.stop_frequency(200e6)
sweeper0.num_points(51)
sweeper0.mapping("linear")
sweeper0.integration_time(100e-6)
sweeper0.num_averages(2)
sweeper0.averaging_mode("sequential")
# print(sweeper0.mapping.__doc__)

In [6]:
sweeper0.run()
result=sweeper0.read()
# sweeper0.plot()

Run a sweep with 51 frequency points in the range of [-2e-10, 200.0] MHz + 5.0 GHz. 
Mapping is linear. 
Integration time = 0.0001 sec. 
Measures 2 times per frequency point. 
Averaging mode is sequential.


## Readout mode of the SHFQA
In Readout mode, **generator** module is used to configure waveform playback, such as upload waveforms, construct and compile readout sequences. Integration parameters and result source are configured by **readout**. Please note that the data type of uploaded waveforms has to be **complex128**.

In [7]:
shfqa.qachannels[0].mode('readout')
generator0 = shfqa.qachannels[0].generator
readout0 = shfqa.qachannels[0].readout

In [8]:
num_readouts = 100
pulse_duration = 100e-9
readout_freq = 200e6
sampling_rate = 2e9

pulse = 0.5*np.exp(2j*np.pi*readout_freq*np.linspace(0, pulse_duration, int(pulse_duration * sampling_rate)))
weight = np.conj(pulse)

shfqa.set_trigger_loopback()
generator0.dig_trigger1_source("chan0trigin0")
# Delay between receving the trigger and playing the readout pulses
generator0.playback_delay(0) 

#Define the program
seqc_program ="""
repeat($param1$) {
    waitDigTrigger(1);
    startQA(QA_GEN_ALL, QA_INT_ALL, true, 0, 0x0);
}    
"""
generator0.set_sequence_params(
    sequence_type="Custom",
    program = seqc_program ,
    custom_params = [num_readouts],
)

# Upload readout pulse and integration weight
generator0.reset_queue()
generator0.queue_waveform(pulse)

generator0.compile()
generator0.upload_waveforms()
readout0.integrations[0].set_int_weights(weight)

Current length of queue: 1
Upload of 1 waveforms took 0.174 s


In [9]:
setup_delay = 200e-9

readout0.integration_length(len(weight))
readout0.integration_delay(setup_delay)
readout0.result_source("result_of_integration")
    
readout0.arm(length = num_readouts, averages = 1)
generator0.stop()
generator0.run()
    
result = readout0.read()

## Scope monitor of the SHFQA

The **scope** module of the SHFQA is used to monitor or record time traces of signals at IF frequency down converted by the SHFQA. 

In [10]:
scope=shfqa.scope
scope.channel1('on')
scope.input_select1
scope.trigger_source("channel0_trigger_input0")
scope.trigger_delay(200e-9)
scope.length(1024)
scope.averaging(1)
scope.segments(1)
scope.run()
result=scope.read()
# scope.stop()

## DIO of the SHFQA
The **dios** of the SHFQA is used to communicate qubit readout results from SHFQA to qubit control instruments, such as SHFSG or HDAWG.

In [11]:
shfqa.nodetree.dio

nodes:
parameters:
 - drive
 - output
 - mode
 - input
 - interface